In [ ]:
import numpy as np
import scipy.io as scio
import librosa
import pathlib
import scipy.io as scio
import pickle
from runSpringerSegmentationAlgorithm import *
from plot_figs import save_figs
from generateSegmentationData import get_data
from undoSegmentationData import get_states

## Load the default options:
# These options control options such as the original sampling frequency of
# the data, the sampling frequency for the derived features and whether the
# mex code should be used for the Viterbi decoding:

springer_options = default_Springer_HSMM_options(SpringerOptions)

## Carrega o modelo
filepath = '/home/linse/stevan/Qualirede/segmentador/springerList.pkl'
pkl_file = open(filepath, 'rb')
model = pickle.load(pkl_file)
pkl_file.close()

## Define os paths
Root = '/media/linse/DADOS/stevan/datasets/heart-sound'
Root = pathlib.Path(Root)

inputPath = Root/'training'
outputPath = Root/'python_segmentation/'
outputPath.mkdir(parents=True, exist_ok=True)

## Coloca as listas em ordem alfabética
folderList = sorted(list(inputPath.glob('training*')))

# 'training-a/', 'training-b/', 'training-c/', 'training-d/', 'training-e/', 'training-f/'
for i in range(len(folderList)):
    
    folderName = folderList[i].name
    
    inFolder = inputPath / folderName
    outFolder = outputPath / folderName
    outFolder.mkdir(parents=True, exist_ok=True)
    
    fileList = sorted(list(inFolder.glob('*.wav')))

    for k in range(len(fileList)):
        
        recordName = inFolder / fileList[k]
        outFile = (outFolder / fileList[k]).with_suffix('.npz')

        print(f'{folderList[i].name} | File {k+1} of {len(fileList)} | {fileList[k].stem}')
            
        # Load data and resample data
        PCG_resampled = librosa.load(recordName,sr=springer_options.audio_Fs)[0]

        # Segmentador
        assigned_states = runSpringerSegmentationAlgorithm(PCG_resampled, model)
        # Transforma vetor de estados para o formato seg_data (dicionário do Python com índices e nome dos estados)
        seg_data = get_data(assigned_states)
        # Salva
        np.savez(outFile, **seg_data)